In [1]:
import pandas as pd
import numpy as np

# 可視化用のライブラリ
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

import neologdn
import MeCab

import re

# テキスト解析
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering

In [2]:
df = pd.read_csv('all_text_wakatigaki.csv',encoding='utf-8')
df.head()

,surface,yomi,original,type,katsuyoukei,katsuyougata,内容
0,neologd,ネオログディー,NEologd,名詞-固有名詞-一般,NaN,NaN,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...
1,_,_,_,記号-一般,NaN,NaN,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...
2,tagger,タガー,tagger,名詞-固有名詞-一般,NaN,NaN,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...
3,.,.,.,記号-一般,NaN,NaN,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...
4,parse,パース,Parse,名詞-固有名詞-一般,NaN,NaN,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...


## テキスト毎にわかち書きされた単語を結合
参考URL：https://github.com/Salinger/found_it_project_06_crwl/blob/master/src/python_crawler_nlp.ipynb

In [3]:
# df.groupbyでグループ化された[内容]毎の[original]に対して文字列のリスト（配列）を一つの文字列に連結する「join」関数を各行（各要素）に対して実行する
# join関数の使い方: '間に挿入する文字列'.join([連結したい文字列のリスト])
text_wakati_df = df.groupby('内容')['original'].apply(lambda x: ' '.join(map(str, x))).reset_index()

# カラム名を適切な名前に変更
text_wakati_df = text_wakati_df.rename(columns={'original':'wakati_words'})

# 確認
text_wakati_df

,内容,wakati_words
0,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,NEologd _ tagger . Parse ( text ) で 各 単語 の 原形 ...
1,まずはparse()で分かち書きした単語群は1つの文字列型になっているので「\n」で区切り、...,まずは Parse () で 分かち書き する た 単語 群 は 1つ の 文字列 型 に ...
2,原形、品詞などの間には「\t」が、分かち書きされた単語と単語の区切りには「\n」が表示される,原形 、 品詞 など の 間 に は 「 \ t 」 が 、 分かち書き する れる た 単...
3,次に邪魔な文字「\t」を省きたい。よって、split('\t')を使用したいがwakatid...,次に 邪魔 だ 文字 「\ t 」 を 省く たい 。 よって 、 split ('\ t ...


## TF-IDFの実装(TfidfVectorizerを使用)
参考URL：https://github.com/hwpwk/NLP/blob/master/TfidfVectorizer%E3%81%AB%E3%82%88%E3%82%8B%E6%96%87%E6%9B%B8%E3%81%AE%E3%83%99%E3%82%AF%E3%83%88%E3%83%AB%E5%8C%96(TF-IDF)%2BDataFrame%E5%8C%96_parse%E3%81%A7%E5%BD%A2%E6%85%8B%E7%B4%A0%E8%A7%A3%E6%9E%90_181030.ipynb

In [4]:
# 初期化
vectorizer= TfidfVectorizer()

# TfidfVectorizerを利用して単語と列番号の対応付けを実行 Document-Term Matrixを獲得できる
tfidf_feature_vectors = vectorizer.fit_transform(text_wakati_df['wakati_words'])

# どの単語を学習しているのかをvectorizer.get_feature_names()で確認できる
# 後にデータフレームを作成する際にカラムとして使用する準備
tfidf_vocabulary = vectorizer.get_feature_names()

## インデックスを元のテキスト、カラムを「わかち書きした単語」にしたデータフレームを作成

In [5]:
tfidf_df = pd.DataFrame(tfidf_feature_vectors.toarray(), index=text_wakati_df['内容'],  columns=tfidf_vocabulary)

# デフォルトだとカラムの表示数が省略されるため最大表示列数の指定（ここでは100列を指定）
pd.set_option('display.max_columns', 100)

# 確認
tfidf_df

,1つ,neologd,parse,split,tagger,text,wakatid,いる,する,たい,ため,できる,ない,など,なる,ので,まずは,よって,れる,リスト,使用,分かち書き,区切り,区切る,単語,原形,品詞,文字,文字列,次に,省く,表示,連続,邪魔,関数
内容,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して表示される,0.000000,0.336187,0.265054,0.000000,0.336187,0.265054,0.000000,0.000000,0.350873,0.000000,0.000000,0.000000,0.000000,0.265054,0.000000,0.000000,0.000000,0.000000,0.265054,0.000000,0.000000,0.000000,0.00000,0.000000,0.214584,0.265054,0.265054,0.000000,0.000000,0.000000,0.000000,0.265054,0.336187,0.000000,0.000000
まずはparse()で分かち書きした単語群は1つの文字列型になっているので「\n」で区切り、リスト型にする,0.316496,0.000000,0.249529,0.000000,0.000000,0.000000,0.000000,0.316496,0.330322,0.000000,0.000000,0.000000,0.000000,0.000000,0.316496,0.316496,0.316496,0.000000,0.000000,0.249529,0.000000,0.249529,0.00000,0.316496,0.202015,0.000000,0.000000,0.000000,0.249529,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
原形、品詞などの間には「\t」が、分かち書きされた単語と単語の区切りには「\n」が表示される,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.341994,0.000000,0.000000,0.000000,0.000000,0.258346,0.000000,0.000000,0.000000,0.000000,0.516693,0.000000,0.000000,0.258346,0.32768,0.000000,0.418307,0.258346,0.258346,0.000000,0.000000,0.000000,0.000000,0.258346,0.000000,0.000000,0.000000
次に邪魔な文字「\t」を省きたい。よって、split('\t')を使用したいがwakatid_textは文字列型ではなくリスト型のためsplit関数を使用できない,0.000000,0.000000,0.000000,0.383927,0.000000,0.151346,0.191963,0.000000,0.100174,0.383927,0.191963,0.191963,0.383927,0.000000,0.000000,0.000000,0.000000,0.191963,0.000000,0.151346,0.383927,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.191963,0.151346,0.191963,0.191963,0.000000,0.000000,0.191963,0.191963
